In [1]:
# librerias a importar

import pandas as pd
pd.set_option('display.max_columns', None)


import warnings
warnings.filterwarnings('ignore')


In [2]:
archivos_csv = ["../data/Anio2019.csv", "../data/Anio2020.csv", "../data/Anio2021.csv", "../data/Anio2022.csv"]

# Lista para almacenar DataFrames de los archivos CSV
dataframes = []

# Lee y carga cada archivo CSV en un DataFrame
for archivo in archivos_csv:
    df = pd.read_csv(archivo, delimiter=";", encoding='latin1')
    dataframes.append(df)

# Concatena los DataFrames en uno solo
resultado = pd.concat(dataframes, ignore_index=True)

# Guarda el resultado en un nuevo archivo CSV
resultado.to_csv("../data/contaminacion.csv", index=False, sep=";")

In [3]:

contaminacion_ori = pd.read_csv('../data/contaminacion.csv', delimiter=";", encoding='latin1') #leo el archivo "contaminacion"
contaminacion = contaminacion_ori.copy() #hago una copia

contaminacion.head() #miro qué pinta tiene para empezar a limpiar

,Fecha,NMT,SituaciÃ¯Â¿Â½n,Ld,Le,Ln,LAeq24,LAS01,LAS10,LAS50,LAS90,LAS99,Nombre
0,ene-19,1,Paseo de Recoletos,"68,5","68,5","65,5","67,8","74,5","71,3","64,9","56,9",49,NaN
1,ene-19,2,Carlos V,"69,9","69,4","66,5","68,9","75,5",71,"67,4","60,7",54,NaN
2,ene-19,3,Plaza del Carmen,"62,7","60,6","60,9","61,8","69,9","63,7","57,5","51,6","49,1",NaN
3,ene-19,4,Plaza de EspaÃ¯Â¿Â½a,"65,5","65,3","61,4","64,5","72,5","66,7","62,8","55,5","47,9",NaN
4,ene-19,5,Barrio del Pilar,"61,6","63,3","57,2",61,"67,2","62,8",58,47,"43,1",NaN


In [4]:
contaminacion.columns #nombre de las columnas

Index(['Fecha', 'NMT', 'SituaciÃ¯Â¿Â½n', 'Ld', 'Le', 'Ln', 'LAeq24', 'LAS01',
       'LAS10', 'LAS50', 'LAS90', 'LAS99', 'Nombre'],
      dtype='object')

In [5]:
contaminacion.info() #info del df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1488 entries, 0 to 1487
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Fecha           1488 non-null   object
 1   NMT             1488 non-null   int64 
 2   SituaciÃ¯Â¿Â½n  372 non-null    object
 3   Ld              1488 non-null   object
 4   Le              1488 non-null   object
 5   Ln              1488 non-null   object
 6   LAeq24          1488 non-null   object
 7   LAS01           1488 non-null   object
 8   LAS10           1488 non-null   object
 9   LAS50           1488 non-null   object
 10  LAS90           1488 non-null   object
 11  LAS99           1488 non-null   object
 12  Nombre          1116 non-null   object
dtypes: int64(1), object(12)
memory usage: 151.3+ KB


In [6]:
columnas_a_eliminar = ["Ld", "Le", "Ln", "LAS01", "LAS10", "LAS50", "LAS90", "LAS99", "Nombre", "NMT", "SituaciÃ¯Â¿Â½n"]

contaminacion = contaminacion.drop(columnas_a_eliminar, axis=1) #elimino toda la info que no me sirve


In [7]:
contaminacion = contaminacion.rename(columns={'LAeq24': 'contam_media'}) #renombro la columna

In [8]:
contaminacion = contaminacion[contaminacion['contam_media'] != "N/D"] #elimino las filas con N/D 

In [9]:
contaminacion.info() #info actual, borré 52 filas de las que no tenía la medición

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1436 entries, 0 to 1487
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Fecha         1436 non-null   object
 1   contam_media  1436 non-null   object
dtypes: object(2)
memory usage: 33.7+ KB


In [10]:
# Divide la columna "Fecha" en "mes" y "año"
contaminacion[['mes', 'año']] = contaminacion['Fecha'].str.split('-', expand=True)

In [11]:
# Mapea los valores del mes a sus equivalentes en número
meses = {
    'ene': '01',
    'feb': '02',
    'mar': '03',
    'abr': '04',
    'may': '05',
    'jun': '06',
    'jul': '07',
    'ago': '08',
    'sep': '09',
    'oct': '10',
    'nov': '11',
    'dic': '12'

}
contaminacion['mes'] = contaminacion['mes'].map(meses)

In [12]:
contaminacion['año'] = '20' + contaminacion['año']  # Agrega "20" antes del año

In [13]:
contaminacion = contaminacion.drop('Fecha', axis=1) # Elimina la columna original "Fecha" 

In [14]:
contaminacion.head() #para ver cómo está quedando

,contam_media,mes,año
0,"67,8",01,2019
1,"68,9",01,2019
2,"61,8",01,2019
3,"64,5",01,2019
4,61,01,2019


In [15]:
# Convierte la columna "contam_media" a tipo numérico (ya que contiene comas en lugar de puntos)
contaminacion['contam_media'] = contaminacion['contam_media'].str.replace(',', '.', regex=True).astype(float)

In [16]:
# Agrupa por mes y calcula la media
contaminacion = contaminacion.groupby(['mes', 'año'])['contam_media'].mean().reset_index().round(1).astype(str)

In [17]:
contaminacion = contaminacion.sort_values(by=['año', 'mes']) #Para ordenar cronológicamente
contaminacion = contaminacion.reset_index(drop=True)

In [18]:
contaminacion = contaminacion[['mes', 'año', 'contam_media']]  # Reordena las columnas

In [19]:
contaminacion.head() #así quedo el df limpito y ordenado

,mes,año,contam_media
0,01,2019,62.7
1,02,2019,61.9
2,03,2019,61.7
3,04,2019,61.6
4,05,2019,61.0


In [20]:
contaminacion.to_csv('contaminacion_limpio.csv', index=False) #lo guardamos en un CSV nuevo


LIMPIEZA ARCHIVO INE 

In [21]:
INE_ori = pd.read_csv('../data/INE.csv', delimiter=";", encoding='latin1') #leo el archivo "INE"

INE = INE_ori.copy() #hago una copia

INE.head() #miro qué pinta tiene para empezar a limpiar

,Total Nacional,Comunidades y Ciudades Autónomas,Edad de la madre,Tipo de dato,Periodo,Total
0,Total Nacional,"13 Madrid, Comunidad de",Todas las edades,Dato base,2022M12,4.307
1,Total Nacional,"13 Madrid, Comunidad de",Todas las edades,Dato base,2022M11,4.461
2,Total Nacional,"13 Madrid, Comunidad de",Todas las edades,Dato base,2022M10,4.513
3,Total Nacional,"13 Madrid, Comunidad de",Todas las edades,Dato base,2022M09,4.439
4,Total Nacional,"13 Madrid, Comunidad de",Todas las edades,Dato base,2022M08,4.423


In [22]:
INE['Total'] = INE['Total'].astype(str).str.replace('.', '').astype(int)
# Agregar un cero al final de los números en las filas con índices 45, 33 y 7 porque al sacarles el punto, perdieron un cero al final del nro
indices_a_actualizar = [45, 33, 7]
for indice in indices_a_actualizar:
    INE.loc[indice, 'Total'] = INE.loc[indice, 'Total'] * 10


In [23]:
INE.head(10)

,Total Nacional,Comunidades y Ciudades Autónomas,Edad de la madre,Tipo de dato,Periodo,Total
0,Total Nacional,"13 Madrid, Comunidad de",Todas las edades,Dato base,2022M12,4307
1,Total Nacional,"13 Madrid, Comunidad de",Todas las edades,Dato base,2022M11,4461
2,Total Nacional,"13 Madrid, Comunidad de",Todas las edades,Dato base,2022M10,4513
3,Total Nacional,"13 Madrid, Comunidad de",Todas las edades,Dato base,2022M09,4439
4,Total Nacional,"13 Madrid, Comunidad de",Todas las edades,Dato base,2022M08,4423
5,Total Nacional,"13 Madrid, Comunidad de",Todas las edades,Dato base,2022M07,4508
6,Total Nacional,"13 Madrid, Comunidad de",Todas las edades,Dato base,2022M06,4642
7,Total Nacional,"13 Madrid, Comunidad de",Todas las edades,Dato base,2022M05,4410
8,Total Nacional,"13 Madrid, Comunidad de",Todas las edades,Dato base,2022M04,4152
9,Total Nacional,"13 Madrid, Comunidad de",Todas las edades,Dato base,2022M03,4329


In [24]:
INE.columns #nombre de las columnas

Index(['Total Nacional', 'Comunidades y Ciudades Autónomas',
       'Edad de la madre', 'Tipo de dato', 'Periodo', 'Total'],
      dtype='object')

In [25]:
columnas_a_eliminar = ['Total Nacional', 'Comunidades y Ciudades Autónomas', 'Edad de la madre', 'Tipo de dato',]

INE = INE.drop(columnas_a_eliminar, axis=1) #elimino toda la info que no me sirve

In [26]:
INE.info()  # para ver la info del df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Periodo  48 non-null     object
 1   Total    48 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 900.0+ bytes


In [27]:
INE[['año', 'mes']] = INE['Periodo'].str.split('M', expand=True)  # Divide la columna "Periodo" en "año" y "mes"
INE = INE.drop('Periodo', axis=1)
INE.head()

,Total,año,mes
0,4307,2022,12
1,4461,2022,11
2,4513,2022,10
3,4439,2022,09
4,4423,2022,08


In [28]:
INE = INE.sort_values(by=['año', 'mes']) #Para ordenar cronológicamente
INE = INE.reset_index(drop=True)

In [29]:
INE = INE[['mes', 'año', 'Total']]  # Reordena las columnas

In [30]:
INE.head() #así queda el df limpio y ordenado

,mes,año,Total
0,01,2019,4705
1,02,2019,4245
2,03,2019,4560
3,04,2019,4699
4,05,2019,4786


In [32]:
INE.to_csv('INE_limpio.csv', index=False) #lo guardamos en un nuevo CSV